In [1]:
from torch_geometric.data import HeteroData, DataLoader
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv, to_hetero , SAGEConv
from torch_geometric.utils import negative_sampling
from torch_geometric.loader import LinkNeighborLoader

import torch
from torch import nn 
import torch.nn.functional as F
import torch.optim as optim

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import LabelEncoder , label_binarize , OneHotEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

import os 
import pandas as pd
import numpy as np
from tqdm import tqdm
from itertools import product
import random
from collections import Counter
import warnings
warnings.filterwarnings("ignore") 


In [29]:
# *****************************************************************************
# Load the Dataframes :
path_work = "/media/concha-eloko/Linux/PPT_clean"
#path_work = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/ficheros_28032023"

    # Open the DF
DF_info = pd.read_csv(f"{path_work}/TropiGATv2.final_df.tsv", sep = "\t" ,  header = 0)


In [57]:
DF_info[DF_info["KL_type_LCA"] == "KL122|KL106"]

,Phage,Protein_name,KL_type_LCA,Infected_ancestor,index,Dataset,seq,domain_seq,1,2,...,1272,1273,1274,1275,1276,1277,1278,1279,1280,prophage_id
1018,GCF_019781185.1__phage14,GCF_019781185.1__phage14__343,KL122|KL106,n4984,anubis__12,anubis,MAYSLVQPSLAGGEISPSLYGRIDLEKYQTSLRRCRNFIVRQSGGI...,KLFYMEQKAVDSVGRWETDKDIGIGDECRYQENFYRCVDGGSNGTT...,0.001141,0.019932,...,-0.063304,0.023525,0.053397,-0.015055,-0.012662,0.054703,-0.113662,-0.111309,0.195060,prophage_5
1030,GCF_012271775.1__phage10,GCF_012271775.1__phage10__12,KL122|KL106,n4984,anubis__12,anubis,MAYSLVQPSLAGGEISPSLYGRIDLEKYQTSLRRCRNFIVRQSGGI...,KLFYMEQKAVDSVGRWETDKDIGIGDECRYQENFYRCVDGGSNGTT...,0.001141,0.019932,...,-0.063304,0.023525,0.053397,-0.015055,-0.012662,0.054703,-0.113662,-0.111309,0.195060,prophage_5
1202,GCF_900500905.1__phage25,GCF_900500905.1__phage25__368,KL122|KL106,n4984,anubis__12,anubis,MAYSLVQPSLAGGEISPSLYGRIDLEKYQTSLRRCRNFIVRQSGGI...,KLFYMEQKAVDSVGRWETDKDIGIGDECRYQENFYRCVDGGSNGTT...,0.001141,0.019932,...,-0.063304,0.023525,0.053397,-0.015055,-0.012662,0.054703,-0.113662,-0.111309,0.195060,prophage_5
1215,GCF_004803085.1__phage3,GCF_004803085.1__phage3__12,KL122|KL106,n4984,anubis__12,anubis,MAYSLVQPSLAGGEISPSLYGRIDLEKYQTSLRRCRNFIVRQSGGI...,KLFYMEQKAVDSVGRWETDKDIGIGDECRYQENFYRCVDGGSNGTT...,0.001141,0.019932,...,-0.063304,0.023525,0.053397,-0.015055,-0.012662,0.054703,-0.113662,-0.111309,0.195060,prophage_5
1236,GCF_900504425.1__phage8,GCF_900504425.1__phage8__368,KL122|KL106,n4984,anubis__12,anubis,MAYSLVQPSLAGGEISPSLYGRIDLEKYQTSLRRCRNFIVRQSGGI...,KLFYMEQKAVDSVGRWETDKDIGIGDECRYQENFYRCVDGGSNGTT...,0.001141,0.019932,...,-0.063304,0.023525,0.053397,-0.015055,-0.012662,0.054703,-0.113662,-0.111309,0.195060,prophage_5
1239,GCF_922831335.1__phage5,GCF_922831335.1__phage5__1337,KL122|KL106,n4984,anubis__12,anubis,MAYSLVQPSLAGGEISPSLYGRIDLEKYQTSLRRCRNFIVRQSGGI...,KLFYMEQKAVDSVGRWETDKDIGIGDECRYQENFYRCVDGGSNGTT...,0.001141,0.019932,...,-0.063304,0.023525,0.053397,-0.015055,-0.012662,0.054703,-0.113662,-0.111309,0.195060,prophage_5
1241,GCF_900500925.1__phage30,GCF_900500925.1__phage30__370,KL122|KL106,n4984,anubis__12,anubis,MAYSLVQPSLAGGEISPSLYGRIDLEKYQTSLRRCRNFIVRQSGGI...,KLFYMEQKAVDSVGRWETDKDIGIGDECRYQENFYRCVDGGSNGTT...,0.001141,0.019932,...,-0.063304,0.023525,0.053397,-0.015055,-0.012662,0.054703,-0.113662,-0.111309,0.195060,prophage_5
1251,GCF_900513485.1__phage8,GCF_900513485.1__phage8__369,KL122|KL106,n4984,anubis__12,anubis,MAYSLVQPSLAGGEISPSLYGRIDLEKYQTSLRRCRNFIVRQSGGI...,KLFYMEQKAVDSVGRWETDKDIGIGDECRYQENFYRCVDGGSNGTT...,0.001141,0.019932,...,-0.063304,0.023525,0.053397,-0.015055,-0.012662,0.054703,-0.113662,-0.111309,0.195060,prophage_5
1316,GCF_001611055.1__phage6,GCF_001611055.1__phage6__1440,KL122|KL106,n4984,anubis__12,anubis,MAYSLVQPSLAGGEISPSLYGRIDLEKYQTSLRRCRNFIVRQSGGI...,KLFYMEQKAVDSVGRWETDKDIGIGDECRYQENFYRCVDGGSNGTT...,0.001141,0.019932,...,-0.063304,0.023525,0.053397,-0.015055,-0.012662,0.054703,-0.113662,-0.111309,0.195060,prophage_5
1327,GCF_016651915.1__phage19,GCF_016651915.1__phage19__13,KL122|KL106,n4984,anubis__12,anubis,MAYSLVQPSLAGGEISPSLYGRIDLEKYQTSLRRCRNFIVRQSGGI...,KLFYMEQKAVDSVGRWETDKDIGIGDECRYQENFYRCVDGGSNGTT...,0.001141,0.019932,...,-0.063304,0.023525,0.053397,-0.015055,-0.012662,0.054703,-0.113662,-0.111309,0.195060,prophage_5


In [56]:
DF_info["KL_type_LCA"].unique()

array(['KL43', 'KL14', 'KL107', 'KL106', 'KL23', 'KL51', 'KL63',
       'KL122|KL106', 'KL30', 'KL64', 'KL47', 'KL36', 'KL15',
       'KL158|KL19|KL15|KL64|KL107|KL52', 'KL31|KL47', 'KL105', 'KL103',
       'KL111', 'KL24', 'KL19',
       'KL19|KL103|KL64|KL107|KL52|KL24|KL62|KL112|KL2', 'KL21', 'KL27',
       'KL2', 'KL102|KL108', 'KL38',
       'KL122|KL25|KL3|KL125|KL18|KL24|KL2|KL158|KL31|KL30|KL35|KL19|KL15|KL21|KL46|KL13|KL110|KL36|KL114|KL28|KL45|KL38|KL55|KL14|KL8|KL107|KL62|KL57|KL39|KL102|KL111|KL16|KL64|KL10|KL108|KL52|KL23|KL60',
       'KL28', 'KL52', 'KL108', 'KL155', 'KL112', 'KL46', 'KL102', 'KL1',
       'KL17', 'KL151', 'KL22', 'KL8', 'KL16', 'KL3', 'KL110', 'KL25',
       'KL163', 'KL10', 'KL48',
       'KL122|KL43|KL25|KL61|KL3|KL151|KL53|KL125|KL117|KL18|KL24|KL58|KL2|KL31|KL30|KL35|KL19|KL15|KL21|KL47|KL106|KL155|KL46|KL13|KL149|KL110|KL114|KL103|KL28|KL142|KL38|KL55|KL14|KL166|KL8|KL20|KL62|KL57|KL1|KL22|KL51|KL39|KL27|KL123|KL102|KL111|KL16|KL64|KL10|KL130|KL113

> ambiguity levels :

In [76]:
# No ambiguity lvl_0: 
DF_info_lvl_0 = DF_info[~DF_info["KL_type_LCA"].str.contains("\\|")]
DF_info_lvl_0 = DF_info_lvl_0.drop_duplicates(subset = ["Infected_ancestor","index","prophage_id"] , keep = "first").reset_index(drop=True)

# ambiguity lvl_1 : tolerate 2 different KLtypes : 
columns_name = DF_info.columns
DF_info_dict = DF_info.to_dict(orient='index')
with open(f"{path_work}/TropiGATv2.ambiguity.lvl_1.tsv", "w") as outfile:
    outfile.write(f"{','.join(columns_name)}\n")
    for index, row in tqdm(DF_info_dict.items()):
        if row["KL_type_LCA"].count("|") == 0:
            line = [str(row[info]) for info in row]
            outfile.write(f"{','.join(line)}\n")
        elif row["KL_type_LCA"].count("|") <= 1:
            kltypes = row["KL_type_LCA"].split("|")
            for index_kl, kltype in enumerate(kltypes) :
                row["KL_type_LCA"] = kltype
                row["Infected_ancestor"] = f"{row['Infected_ancestor']}_{index_kl}"
                line = [str(row[info]) for info in row]
                outfile.write(f"{','.join(line)}\n")
        else :
            pass

# ambiguity lvl_2 : tolerate 3 different KLtypes : 
columns_name = DF_info.columns
DF_info_dict = DF_info.to_dict(orient='index')
with open(f"{path_work}/TropiGATv2.ambiguity.lvl_2.tsv", "w") as outfile:
    outfile.write(f"{','.join(columns_name)}\n")
    for index, row in tqdm(DF_info_dict.items()):
        if row["KL_type_LCA"].count("|") == 0:
            line = [str(row[info]) for info in row]
            outfile.write(f"{','.join(line)}\n")
        elif row["KL_type_LCA"].count("|") <= 2:
            kltypes = row["KL_type_LCA"].split("|")
            for index_kl, kltype in enumerate(kltypes) :
                row["KL_type_LCA"] = kltype
                row["Infected_ancestor"] = f"{row['Infected_ancestor']}_{index_kl}"
                line = [str(row[info]) for info in row]
                outfile.write(f"{','.join(line)}\n")
        else :
            pass


100%|███████████████████████████████████| 20139/20139 [00:14<00:00, 1418.85it/s]


In [59]:
# level 0 :
DF_info_lvl_0 = DF_info[~DF_info["KL_type_LCA"].str.contains("\\|")]
DF_info_lvl_0 = DF_info_lvl_0.drop_duplicates(subset = ["Infected_ancestor","index","prophage_id"] , keep = "first").reset_index(drop=True)

# level 1 :
DF_info_lvl_1 = pd.read_csv(f"{path_work}/TropiGATv2.ambiguity.lvl_1.tsv", sep = "," ,  header = 0)
DF_info_lvl_1 = DF_info_lvl_1.drop_duplicates(subset = ["Infected_ancestor","index","prophage_id"] , keep = "first").reset_index(drop=True)

# level 2 :
DF_info_lvl_2 = pd.read_csv(f"{path_work}/TropiGATv2.ambiguity.lvl_2.tsv", sep = "," ,  header = 0)
DF_info_lvl_2 = DF_info_lvl_2.drop_duplicates(subset = ["Infected_ancestor","index","prophage_id"] , keep = "first").reset_index(drop=True)


In [37]:
DF_info_lvl_0

,Phage,Protein_name,KL_type_LCA,Infected_ancestor,index,Dataset,seq,domain_seq,1,2,...,1272,1273,1274,1275,1276,1277,1278,1279,1280,prophage_id
0,GCF_016651625.1__phage29,GCF_016651625.1__phage29__142,KL43,GCF_016651625.1,ppt__1,ppt,MSVPNQTPYNIYTANGLTTVFTYEFYIISASDLRVSINGDVVTSGY...,KDFVNINDYWFPTDGDDFYPALNKALSVSPHVLIPPGKHYLKSTVS...,-0.018416,0.022387,...,0.004437,0.087907,0.015800,0.025778,0.065790,0.034045,-0.070899,0.016068,0.065339,prophage_12186
1,GCF_016651625.1__phage29,GCF_016651625.1__phage29__150,KL43,GCF_016651625.1,anubis__0,anubis,MRANLIKTNFTAGEISPRLMGRVDIARYANGAKIIENAVCVVQGGV...,QAASPGAWTREDTVWTEEFGYPGAVTLYQQRLVLAGSPQYPQTIWW...,0.036016,0.005938,...,-0.037612,0.008772,0.010556,-0.049738,-0.012549,0.092624,-0.136602,-0.191378,0.135658,prophage_12186
2,GCF_016651625.1__phage12,GCF_016651625.1__phage12__59,KL43,GCF_016651625.1,ppt__4,ppt,MSISKRNFLKAVSCAYFFYSFKALTKVNQPIEDYISTKDKNTWPSK...,NTWPSKVHRVEEFYTSTDRDYSDAILRGINYCSLNNCVLFFSDKYK...,0.026004,0.024372,...,-0.026018,0.018206,0.036751,-0.032549,0.064112,0.061520,-0.024423,-0.027998,0.028089,prophage_924
3,GCF_019928025.1__phage0,GCF_019928025.1__phage0__10,KL43,n1471,ppt__4,ppt,MSISKRNFLKAVSCAYFFYSFKALTKVNQPIEDYISTKDKNTWPSK...,NTWPSKVHRVEEFYTSTDRDYSDAILRGINYCSLNNCVLFFSDKYK...,0.026004,0.024372,...,-0.026018,0.018206,0.036751,-0.032549,0.064112,0.061520,-0.024423,-0.027998,0.028089,prophage_2929
4,GCF_004313505.1__phage4,GCF_004313505.1__phage4__113,KL14,GCF_004313505.1,anubis__5,anubis,MSEYDTGNPVPSASMPDAWDNMQSIDKFVNSSDETITTRTGQQLDT...,KAIFDAWLDFGIDWNGNESISLQLQTAVNYVSKLPYGGEIVLRPGV...,-0.023648,0.052674,...,-0.025991,0.068538,-0.051192,0.026481,0.069100,0.017813,-0.103797,0.018961,0.117058,prophage_11091
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11097,GCF_002248635.1__phage4,GCF_002248635.1__phage4__44,KL102,n320,anubis_return__4264,anubis_return,MVSLKGMGSTFRDCTALISLPSGLLDGCINLTSLTLTFSGCTSLAL...,MVSLKGMGSTFRDCTALISLPSGLLDGCINLTSLTLTFSGCTSLAL...,-0.000585,-0.087093,...,0.036749,0.048489,0.020484,0.023950,-0.048109,0.134457,-0.101326,0.088485,0.037368,prophage_3054
11098,GCF_001905235.1__phage21,GCF_001905235.1__phage21__0,KL107,n35403540,anubis_return__4272,anubis_return,MLKHSLAIATCLAFSSSVMGNEANLLYTNTMQFPYKHNADGYMVFD...,VMGNEANLLYTNTMQFPYKHNADGYMVFDIHGKLVVPPEGHFDTLN...,0.076721,0.027635,...,0.042391,-0.004292,-0.004047,-0.011631,-0.026469,0.070159,-0.077212,-0.077950,-0.034630,prophage_313
11099,GCF_004312845.1__phage3,GCF_004312845.1__phage3__38,KL9,GCF_004312845.1,anubis_return__4275,anubis_return,MAILITGKSMTRLPESSSWEEEIELITRSERVAGGLDGPANRPLKS...,DAVIRRDLASDKGTSGVGKLGDKPLVAISYYKSKGQSDQDAVQAAF...,0.032196,0.048856,...,-0.016331,0.084711,0.056063,0.001793,0.073958,0.090169,-0.060105,0.023726,0.086452,prophage_12656
11100,GCF_900172635.1__phage2,GCF_900172635.1__phage2__1608,KL124,GCF_900172635.1,anubis_return__4287,anubis_return,MADLSISVISDQASESNQAGWWHPLDSFQGVEYYGLCKEYGTAGYH...,MADLSISVISDQASESNQAGWWHPLDSFQGVEYYGLCKEYGTAGYH...,-0.011089,-0.005328,...,0.034656,0.046130,0.012586,-0.021702,-0.023386,0.105700,-0.099147,-0.057367,0.091427,prophage_12780


***
# Build the graph :

In [67]:
def make_graph(df_info) : 
    # **************************************************************
    # initialize the graph
    graph_data = HeteroData()
    # Indexation process (shall add the N phages to predict)
    indexation_nodes_A = df_info["Infected_ancestor"].unique().tolist()  
    indexation_nodes_B1 = df_info["Phage"].unique().tolist()
    indexation_nodes_B2 = df_info["index"].unique().tolist() 
    ID_nodes_A = {item:index for index, item in enumerate(indexation_nodes_A)}
    ID_nodes_A_r = {index:item for index, item in enumerate(indexation_nodes_A)}
    ID_nodes_B1 = {item:index for index, item in enumerate(indexation_nodes_B1)}
    ID_nodes_B1_r = {index:item for index, item in enumerate(indexation_nodes_B1)}
    ID_nodes_B2 = {item:index for index, item in enumerate(indexation_nodes_B2)}
    ID_nodes_B2_r = {index:item for index, item in enumerate(indexation_nodes_B2)}
    # **************************************************************
    # Make the node feature file : 
    OHE = OneHotEncoder(sparse=False)
    one_hot_encoded = OHE.fit_transform(df_info[["KL_type_LCA"]])
    label_mapping = {label: one_hot_encoded[i] for i, label in enumerate(OHE.categories_[0])}
    embeddings_columns = [str(i) for i in range(1, 1281)]
    node_feature_A = torch.tensor([label_mapping[df_info[df_info["Infected_ancestor"] == ID_nodes_A_r[i]]["KL_type_LCA"].values[0]] for i in range(0,len(ID_nodes_A_r))], dtype=torch.float)
    node_feature_B1 = torch.zeros((len(ID_nodes_B1), 0), dtype=torch.float)
    node_feature_B2 = torch.tensor([df_info[df_info["index"] == ID_nodes_B2_r[i]][embeddings_columns].values[0].tolist() for i in range(0,len(ID_nodes_B2_r))] , dtype=torch.float)
    # feed the graph
    graph_data["A"].x = node_feature_A
    graph_data["B1"].x = node_feature_B1
    graph_data["B2"].x = node_feature_B2
    # **************************************************************
    # Make edge file
    # Node B1 (prophage) - Node A (bacteria) :
    edge_index_B1_A = []
    for _, row in df_info.iterrows() :
        edge_index_B1_A.append([ID_nodes_B1[row["Phage"]], ID_nodes_A[row["Infected_ancestor"]]])
    edge_index_B1_A = torch.tensor(edge_index_B1_A , dtype=torch.long)
    # Node A (bacteria) - Node B1 (prophage) :
    edge_index_A_B1 = []
    for _, row in df_info.iterrows() :
        edge_index_A_B1.append([ID_nodes_A[row["Infected_ancestor"]] , ID_nodes_B1[row["Phage"]]])
    edge_index_A_B1 = torch.tensor(edge_index_A_B1 , dtype=torch.long)
    # Node B2 (depolymerase) - Node B1 (prophage) :
    edge_index_B2_B1 = []
    for phage in df_info.Phage.unique() :
        all_data_phage = df_info[df_info["Phage"] == phage]
        for _, row in all_data_phage.iterrows() :
            edge_index_B2_B1.append([ID_nodes_B2[row["index"]], ID_nodes_B1[row["Phage"]]])
    edge_index_B2_B1 = torch.tensor(edge_index_B2_B1 , dtype=torch.long)
    # feed the graph
    graph_data['B1', 'infects', 'A'].edge_index = edge_index_B1_A.t().contiguous()
    graph_data['B2', 'expressed', 'B1'].edge_index = edge_index_B2_B1.t().contiguous()
    # That one is optional  
    graph_data['A', 'harbors', 'B1'].edge_index = edge_index_A_B1.t().contiguous()
    # **************************************************************
    # Make the Y file : 
    graph_data['B1', 'infects', 'A'].y = torch.ones(len(graph_data['B1', 'infects', 'A'].edge_index[0]))
    graph_data['B2', 'expressed', 'B1'].y = torch.ones(len(graph_data['B2', 'expressed', 'B1'].edge_index[0]))
    # That one is optional  
    graph_data['A', 'harbors', 'B1'].y = torch.ones(len(graph_data['A', 'harbors', 'B1'].edge_index[0]))
    
    return graph_data
    

In [79]:
graph_lvl_0 = make_graph(DF_info_lvl_0)
torch.save(graph_lvl_0 , f'{path_work}/Tropi_graph.lvl_0.1309.pt')

In [77]:
graph_lvl_1 = make_graph(DF_info_lvl_1)
torch.save(graph_lvl_1 , f'{path_work}/Tropi_graph.lvl_1.1309.pt')

In [78]:
graph_lvl_2 = make_graph(DF_info_lvl_2)
torch.save(graph_lvl_2 , f'{path_work}/Tropi_graph.lvl_2.1309.pt')

In [80]:
graph_lvl_0

HeteroData(
  A={ x=[5069, 129] },
  B1={ x=[8815, 0] },
  B2={ x=[4105, 1280] },
  (B1, infects, A)={
    edge_index=[2, 11102],
    y=[11102]
  },
  (B2, expressed, B1)={
    edge_index=[2, 11102],
    y=[11102]
  },
  (A, harbors, B1)={
    edge_index=[2, 11102],
    y=[11102]
  }
)

In [81]:
graph_lvl_1

HeteroData(
  A={ x=[5120, 129] },
  B1={ x=[8900, 0] },
  B2={ x=[4139, 1280] },
  (B1, infects, A)={
    edge_index=[2, 11200],
    y=[11200]
  },
  (B2, expressed, B1)={
    edge_index=[2, 11200],
    y=[11200]
  },
  (A, harbors, B1)={
    edge_index=[2, 11200],
    y=[11200]
  }
)

In [82]:
graph_lvl_2

HeteroData(
  A={ x=[5291, 129] },
  B1={ x=[8962, 0] },
  B2={ x=[4167, 1280] },
  (B1, infects, A)={
    edge_index=[2, 11532],
    y=[11532]
  },
  (B2, expressed, B1)={
    edge_index=[2, 11532],
    y=[11532]
  },
  (A, harbors, B1)={
    edge_index=[2, 11532],
    y=[11532]
  }
)

In [ ]:
rsync -avzhe ssh \
/media/concha-eloko/Linux/PPT_clean/Tropi_graph.lvl_0.1309.pt \
conchae@garnatxa.srv.cpd:/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/ficheros_28032023/train_nn

rsync -avzhe ssh \
/media/concha-eloko/Linux/PPT_clean/Tropi_graph.lvl_1.1309.pt \
conchae@garnatxa.srv.cpd:/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/ficheros_28032023/train_nn

rsync -avzhe ssh \
/media/concha-eloko/Linux/PPT_clean/Tropi_graph.lvl_2.1309.pt \
conchae@garnatxa.srv.cpd:/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/ficheros_28032023/train_nn
